In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json

import joblib
import torch

import numpy as np
import pandas as pd
import xgboost as xgb

from scipy.stats import pearsonr

from IPython.display import Image

import plotly.graph_objects as go

from utils import scale_data
from utils import preprocess_CH_data
from utils import set_seeds
from utils import kriging_ch

from utils import bayes_filter
from utils import l2norm_km
from utils import print_metrics

from utils import train_teleport
from utils import test_models

In [ ]:
from constants import LOCATION_MAPPING_CH
from constants import MIN_LON_CH
from constants import MAX_LON_CH
from constants import MIN_LAT_CH
from constants import MAX_LAT_CH
from constants import SIZE_X
from constants import SIZE_Y

from constants import SEED

from constants import NUM_ESTIMATORS

In [ ]:
pd.options.mode.chained_assignment = None 

In [ ]:
SENSOR_COLS = ['O3', 'YTD']
TARGET = ['TEMP']
X_WEST = 70
X_EAST = 140
CH_DATA = os.path.abspath('data/ch_data/*.csv')
FN_MODEL_TELEPORT_OZONE = 'assets/ozone_models_teleport.pkl'
FN_MODEL_H_OZONE = 'assets/ozone_models_H.pkl'

# Prepare Data

In [ ]:
set_seeds()

In [ ]:
df = preprocess_CH_data(CH_DATA)

df['datetime'] = pd.to_datetime(df['Datum/Zeit'], format='%d.%m.%Y')
df, scaler = scale_data(df, SENSOR_COLS + TARGET)

kriged_maps = {}
for key in SENSOR_COLS + TARGET:
    if key == 'YTD':
        continue
    
    kriged_maps[key] = kriging_ch(df, key)
    grid_x = np.arange(MIN_LON_CH, MAX_LON_CH, (MAX_LON_CH - MIN_LON_CH) / SIZE_X)
    grid_y = np.arange(MIN_LAT_CH, MAX_LAT_CH, (MAX_LAT_CH - MIN_LAT_CH) / SIZE_Y)

    for station in df['station'].unique():
        lon = LOCATION_MAPPING_CH.loc[LOCATION_MAPPING_CH['station'] == station]['lon'].item()
        lat = LOCATION_MAPPING_CH.loc[LOCATION_MAPPING_CH['station'] == station]['lat'].item()
        real_x = np.abs(grid_x - lon).argmin()
        real_y = np.abs(grid_y - lat).argmin()
        df.loc[df['station'] == station, 'lon'] = lon
        df.loc[df['station'] == station, 'lat'] = lat
        df.loc[df['station'] == station, 'real_x'] = real_x
        df.loc[df['station'] == station, 'real_y'] = real_y
df = df[['datetime', 'station'] + SENSOR_COLS + TARGET + ['lon', 'lat', 'real_x', 'real_y']]

In [ ]:
for station in df['station'].unique():
    for x in [-1, 0, 1]:
        for y in [-1, 0, 1]:
            if x == 0 and y == 0:
                continue
                            
            tmp_df = df.loc[df['station'] == station]
            
            new_x = int(tmp_df['real_x'].iloc[0] + x)
            new_y = int(tmp_df['real_y'].iloc[0] + y)
            tmp_df['real_x'] = new_x
            tmp_df['real_y'] = new_y
            tmp_df['station'] =  tmp_df['station'].iloc[0] + '_' + str(x) + '_' + str(y)
            
            for k in kriged_maps.keys():
                tmp_df[k] = kriged_maps[k][:, new_x, new_y]
                                
            df = pd.concat([df, tmp_df])
df = df.reset_index().drop(columns=['index'])

In [ ]:
n_examples_data = len(df)
val_examples_data = np.random.choice(df.index, int(n_examples_data * 0.3), replace=False)
df_val_data = df.loc[df['station'].isin(df.iloc[val_examples_data]['station'].unique())]
df_train_data = df.drop(val_examples_data)

n_examples_station = len(df['station'].unique())
val_examples_station = np.random.choice(df['station'].unique(), int(n_examples_station * 0.3), replace=False)
df_val_station = df.loc[(df['station'].isin(val_examples_station))]
df_train_station = df.loc[(~df['station'].isin(val_examples_station))]

df_joined = df.query(f'real_x < {X_WEST} or real_x > {X_EAST}')
n_examples_west = len(df_joined.query(f'real_x < {X_WEST}')['station'].unique())
n_examples_east = len(df_joined.query(f'real_x > {X_EAST}')['station'].unique())
val_examples_west = np.random.choice(df_joined['station'].unique(), int(n_examples_west * 0.3), replace=False)
val_examples_east = np.random.choice(df_joined['station'].unique(), int(n_examples_east * 0.3), replace=False)

In [ ]:
print(f'Total Stations: {n_examples_station:22}')
print(f'Station Split Validation Stations: {len(val_examples_station):3}')
print(f'Station Split Traning Stations: {n_examples_station-len(val_examples_station):6}')

# Training
## Train Model H

In [ ]:
if not os.path.exists(FN_MODEL_H_OZONE):
    models_H = {}
    for n in ['data_split', 'station_split']:
        if n == 'data_split':
            X = df_train_data[SENSOR_COLS]
            y = df_train_data[TARGET]
        elif n == 'station_split':
            X = df_train_station[SENSOR_COLS]
            y = df_train_station[TARGET]

        reg = xgb.XGBRegressor(n_estimators=NUM_ESTIMATORS, random_state=SEED, n_jobs=8)
        reg.fit(X, y)

        models_H[n]= reg
    joblib.dump(models_H, FN_MODEL_H_OZONE)
else:
    models_H = joblib.load(FN_MODEL_H_OZONE)

In [ ]:
dist = {}

for n in models_H:
    if n == 'data_split':
        df_val = df_val_data
    elif n == 'station_split':
        df_val = df_val_station
    
    dist[n] = print_metrics(df_val, 'station', models_H[n], SENSOR_COLS, TARGET, kriged_maps, scaler, 'S')

## Train Teleport Models

In [ ]:
df_train_west = df_joined.loc[~df_joined['station'].isin(val_examples_west)].query(f'real_x < {X_WEST}')
df_train_east = df_joined.loc[~df_joined['station'].isin(val_examples_east)].query(f'real_x > {X_EAST}')
df_val_west = df_joined.loc[(df_joined['station'].isin(val_examples_west))]
df_val_east = df_joined.loc[(df_joined['station'].isin(val_examples_east))]

In [ ]:
if not os.path.exists(FN_MODEL_TELEPORT_OZONE):
    models_T = train_teleport(
        df_train_west[SENSOR_COLS].to_numpy(),
        df_train_east[SENSOR_COLS].to_numpy(),
        df_val_west[SENSOR_COLS].to_numpy(),
        df_val_east[SENSOR_COLS].to_numpy(),
    )
    joblib.dump(models_T, FN_MODEL_TELEPORT_OZONE)
else:
    models_T = joblib.load(FN_MODEL_TELEPORT_OZONE)
    
enc_a = models_T['enc_a']
enc_b = models_T['enc_b']
lat = models_T['lat']
dec_a = models_T['dec_a']
dec_b = models_T['dec_b']

# Test

In [ ]:
orig_a, tele_a, orig_b, tele_b, rse_tele_a, rse_tele_b, mae_ae = test_models(
    'station', df_val_west, df_val_east, SENSOR_COLS, 
    models_H['data_split'], models_T,
    kriged_maps['TEMP'], scaler, 'S'
)

In [ ]:
mae_ae = np.array(mae_ae)
print(f'AutoEncoder MAE: {mae_ae.mean()}')

In [ ]:
distance_ew = l2norm_km((X_WEST, 0), (X_EAST, 0), 'S')
print(f'Distance East-West: {distance_ew}')

# Plots

In [ ]:
df_val = pd.concat([df_val_west, df_val_east]).reset_index().drop(columns=['index'])

## Scatter Plot Real vs Predicted Values (Model H)

In [ ]:
orig = []
pred = []

for _, tmp_df in df_val.groupby(['station']):
    real_scaled = scaler.inverse_transform(tmp_df['TEMP'].to_numpy(), ['TEMP']).ravel()

    w_prime = models_H['data_split'].predict(tmp_df[SENSOR_COLS])
    w_prime_scaled = scaler.inverse_transform(w_prime, ['TEMP']).ravel()

    pred.extend(w_prime_scaled + 273.15)
    orig.extend(real_scaled + 273.15)
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=orig, y=np.array(pred).ravel(), mode='markers', name='Prediction'))

fig.add_trace(go.Scatter(x=[260, 300], y=[260,300], mode='lines'))
fig.update_layout(showlegend=False)

fig.update_layout(autosize=False, width=500, height=500, font={'size': 24}, template='simple_white')
fig.update_layout(margin={'l': 0,'r': 0, 'b': 0,'t': 0})
fig.update_layout(xaxis_title='Real Temperature (K)', yaxis_title='Predicted Temperature (K)')
img = fig.to_image(format="png")
with open('plots/ozone_weather_scatter.png', 'wb') as f:
    f.write(img)
Image(img)

##  Localisation over Time

In [ ]:
df_results = pd.DataFrame()
durations = [7, 14, 31, 90, 180, 366]
arguments = []

for _, tmp_df in df_val.groupby(['station']):
    real_x = int(tmp_df['real_x'].iloc[0])
    real_y = int(tmp_df['real_y'].iloc[0])

    for duration in durations:
        for run in range(20):
            if duration != 366:
                max_time = kriged_maps['TEMP'].shape[0]
                start = np.random.randint(0, max_time - duration)
            else:
                if duration == 366 and run == 0:
                    start = 0
                else:
                    break

            w_prime = models_H['data_split'].predict(tmp_df[SENSOR_COLS])
            w_prime_range = w_prime[start:start+duration]
            x, y = bayes_filter(w_prime_range, kriged_maps['TEMP'], start, duration)
            distance = l2norm_km((real_x, real_y), (x, y), 'S')
            result = {'station': station, 'real_x': real_x, 'real_y': real_y, 'pred_x': x, 'pred_y': y,
                      'distance': distance, 'start': start, 'duration': duration, 'run': run}
            df_results = df_results.append(pd.DataFrame(result, index=[0]))

In [ ]:
fig = go.Figure()
for dur in df_results['duration'].unique():
    tmp_df = df_results.loc[df_results['duration']==dur]
    fig.add_trace(go.Box(y=tmp_df['distance'], name=int(dur)))
fig.update_layout(showlegend=False, template='simple_white', margin={'l': 0,'r': 0, 'b': 0,'t': 0})
fig.update_layout(xaxis_title='Days', font={'size': 24})
fig.update_yaxes(title_text='Error (km)', range=[0, 50])
fig.show()

fig.write_image('plots/ozone_localization.pdf')

## Map of Switzerland

In [ ]:
import json
from constants import LOCATION_MAPPING_CH

with open('assets/switzerland.geojson', 'r') as f:
    swiss = json.load(f)

marker_west = {'color': 'blue', 'size': 20}
marker_east = {'color': 'red', 'size': 20}
marker_middle = {'color': 'grey', 'size': 20}
    
df_stations = LOCATION_MAPPING_CH.loc[LOCATION_MAPPING_CH['station'].isin(df['station'].unique())]

fig = go.Figure()

fig = go.Figure()
# add stations on the west
fig.add_trace(go.Scattermapbox(lat=df[df.real_x < X_WEST].groupby('station').mean()['lat'],
                               lon=df[df.real_x < X_WEST].groupby('station').mean()['lon'],
                               marker=marker_west,
                               name = "West")
                               )
# add stations in the middle
fig.add_trace(go.Scattermapbox(lat=df[(df.real_x < X_EAST) & (df.real_x > X_WEST)].groupby('station').mean()['lat'],
                               lon=df[(df.real_x < X_EAST) & (df.real_x > X_WEST)].groupby('station').mean()['lon'],
                               marker=marker_middle,
                               name = "Middle")
                               )

# add stations on the east
fig.add_trace(go.Scattermapbox(lat=df[df.real_x > X_EAST].groupby('station').mean()['lat'],
                               lon=df[df.real_x > X_EAST].groupby('station').mean()['lon'],
                               marker=marker_east,
                               name = "East")
                               )

fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    mapbox=go.layout.Mapbox(
        style="stamen-terrain", 
        zoom=6.9,
        center_lat =  46.79,
        center_lon = 8.23,
        layers=[{
            'sourcetype': 'geojson',
            'source': swiss,
            'type': 'line',
        }]
    )
)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.update_layout(mapbox_style='carto-positron')
fig.show()

## Original vs Teleported

In [ ]:
tmp_df = df_val.loc[df_val['station'] == 'Magadino-Cadenazzo_0_-1']
S_ab = torch.Tensor(tmp_df[SENSOR_COLS].to_numpy())
with torch.no_grad():
    s_tele_a = dec_b(lat(enc_a(S_ab)))
S_ab = S_ab.numpy()
s_tele_a = s_tele_a.numpy()

fig = go.Figure()
fig.add_trace(go.Scatter(y=scaler.inverse_transform(S_ab[:, 0], [SENSOR_COLS[0]]).squeeze() + 273.15, name='Original', line=dict(width=3)))
fig.add_trace(go.Scatter(y=scaler.inverse_transform(s_tele_a[:, 0], [SENSOR_COLS[0]]).squeeze() + 273.15, name='Teleported'))
fig.update_layout(template='simple_white', margin={'l': 0,'r': 0, 'b': 0,'t': 0})
fig.update_layout(font={'size': 24}, xaxis_title='Days', yaxis_title='Ozone (µg/m³)')
fig.update_layout(legend={'orientation': 'h', 'yanchor': 'bottom', 'y': 1.02, 'xanchor': 'right', 'x': 1})
fig.show()
fig.write_image('plots/ozone_teleport_sample.pdf')

In [ ]:
pearsonr(S_ab[:, 0], s_tele_a[:, 0])

## Distance from Teleported

In [ ]:
stations_west = df_val_west['station'].unique()
stations_east = df_val_east['station'].unique()
station_distances = []

for i in range(len(stations_west)):
    tmp_df = df_val.loc[df_val['station'] == stations_west[i]]
    x_A = tmp_df['real_x'].iloc[0]
    y_A = tmp_df['real_y'].iloc[0]
    for i in range(len(stations_east)):
        tmp_df = df_val.loc[df_val['station'] == stations_east[i]]
        x_B = tmp_df['real_x'].iloc[0]
        y_B = tmp_df['real_y'].iloc[0]
        
        station_distances.append(l2norm_km((x_A, y_A), (x_B, y_B), 'S'))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=station_distances, name='Station Distances'))
fig.add_trace(go.Box(y=orig_a, name='Original Trace West'))
fig.add_trace(go.Box(y=tele_a, name='Teleported Trace West'))
fig.add_trace(go.Box(y=orig_b, name='Original Trace East'))
fig.add_trace(go.Box(y=tele_b, name='Teleported Trace East'))
fig.update_layout(showlegend=False, template='simple_white', margin={'l': 0,'r': 0, 'b': 0,'t': 0})
fig.update_layout(font={'size': 24}, yaxis_title='Error (km)')
fig.show()

fig.write_image('plots/ozone_teleport_localization.pdf')

In [ ]:
avg_err = (np.abs(np.array(orig_a) - np.array(tele_a)).mean() + np.abs(np.array(orig_b) - np.array(tele_b)).mean())/2
print(f'Average Error : {avg_err} km')

In [ ]:
err_a = np.abs(np.array(orig_a) - np.array(tele_a)).mean()
err_b = np.abs(np.array(orig_b) - np.array(tele_b)).mean()
print(f'Error A: {err_a} km Err B: {err_b} km')

In [ ]:
rel_error = ((np.array(tele_a) - np.array(orig_a)) / np.abs(np.array(tele_a))).mean() * 100
print(f'Relative Error A: {rel_error}%')
rel_error = ((np.array(tele_b) - np.array(orig_b)) / np.abs(np.array(tele_b))).mean() * 100
print(f'Relative Error B: {rel_error}%')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=np.array(rse_tele_a), name='Teleported Trace West'))
fig.add_trace(go.Box(y=np.array(rse_tele_b), name='Teleported Trace East'))
fig.update_layout(showlegend=False, template='simple_white', margin={'l': 0,'r': 0, 'b': 0,'t': 0})
fig.update_layout(font={'size': 24}, yaxis_title='Mean Absolute Error')
fig.update_yaxes(range=[0, 0.12])
fig.show()

fig.write_image('plots/ozone_teleport_mae.pdf')